In [2]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd     
import matplotlib
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("RNA_matrix_breastCancerOnly.csv")


In [6]:
df.head()

,file_id,label,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,...,ENSG00000243547.1,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1
0,GTEX-1117F-0226-SM-5GZZ7,Healthy,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.0,1.0,469.0,0.0,0.0,0.0,15.0,7.0,0.0,0.0
1,GTEX-111CU-1826-SM-5GZYN,Healthy,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,...,26.0,0.0,1055.0,0.0,0.0,0.0,21.0,6.0,0.0,0.0
2,GTEX-111FC-0226-SM-5N9B8,Healthy,0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,...,37.0,0.0,1027.0,0.0,0.0,0.0,24.0,6.0,0.0,0.0
3,GTEX-111VG-2326-SM-5N9BK,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,997.0,0.0,0.0,0.0,25.0,2.0,0.0,0.0
4,GTEX-111YS-2426-SM-5GZZQ,Healthy,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,20.0,0.0,945.0,0.0,0.0,0.0,14.0,1.0,0.0,0.0


In [9]:
df.drop(["file_id", "label"], "columns", inplace=True)

In [10]:
df.head()

,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,ENSG00000265820.1,ENSG00000108947.4,...,ENSG00000243547.1,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1
0,0.0,1554.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,88.0,...,17.0,0.0,2112.0,0.0,0.0,0.0,9.0,3.0,0.0,0.0
1,0.0,751.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,339.0,...,21.0,0.0,1727.0,0.0,0.0,0.0,24.0,6.0,0.0,0.0
2,0.0,169.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,314.0,...,2.0,0.0,851.0,0.0,0.0,2.0,15.0,2.0,0.0,0.0
3,0.0,321.0,1.0,9.0,1.0,0.0,0.0,0.0,2.0,251.0,...,84.0,0.0,2157.0,0.0,0.0,0.0,12.0,3.0,0.0,0.0
4,0.0,1921.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,281.0,...,31.0,2.0,1856.0,0.0,0.0,0.0,26.0,2.0,0.0,0.0


In [11]:
from sklearn.cluster import KMeans
k_means = KMeans(n_clusters=5)
k_means.fit(df) 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=7, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [19]:
results = k_means.labels_
np.unique(results)

array([0, 1, 2, 3, 4], dtype=int32)

In [20]:
df["clusterGroup"] = np.nan
dataLength = len(df)
for i in range(dataLength):
    df.iloc[i, df.columns.get_loc("clusterGroup")] = results[i]

In [21]:
df_clusterData = df.groupby(["clusterGroup"]).mean()
df_clusterData

,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,ENSG00000265820.1,ENSG00000108947.4,...,ENSG00000243547.1,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1
clusterGroup,,,,,,,,,,,,,,,,,,,,,
0.0,0.145833,783.821181,0.097222,1.460069,2.406250,0.159722,0.039931,0.008681,0.666667,380.815972,...,30.225694,0.649306,1802.510417,0.329861,0.052083,0.164931,35.890625,6.215278,0.032986,0.046875
1.0,0.102426,874.142857,0.145553,2.328841,2.514825,0.180593,0.021563,0.002695,0.830189,414.560647,...,35.331536,0.695418,1902.959569,0.172507,0.037736,0.221024,42.194070,7.229111,0.094340,0.010782
2.0,0.000000,790.571429,0.214286,2.285714,1.071429,0.142857,0.000000,0.000000,0.571429,221.428571,...,16.000000,0.500000,1268.071429,0.285714,0.000000,0.071429,63.500000,7.571429,0.000000,0.000000
3.0,0.094203,985.043478,0.065217,1.731884,2.188406,0.239130,0.021739,0.000000,0.724638,392.891304,...,36.434783,0.500000,1847.014493,0.217391,0.181159,0.217391,43.920290,8.181159,0.007246,0.007246
4.0,0.500000,639.750000,1.250000,4.250000,517.000000,1.500000,0.250000,0.500000,15.000000,111.000000,...,28.750000,4.750000,1078.000000,0.000000,0.000000,1.000000,22.500000,25.000000,0.000000,0.000000


In [22]:
df_clusterData["Count"] = pd.Series(df.groupby(["clusterGroup"]).size())
df_clusterData.insert(0, 'Count_', df_clusterData.Count)

In [23]:
df_clusterData

,Count_,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,ENSG00000265820.1,...,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1,Count
clusterGroup,,,,,,,,,,,,,,,,,,,,,
0.0,576,0.145833,783.821181,0.097222,1.460069,2.406250,0.159722,0.039931,0.008681,0.666667,...,0.649306,1802.510417,0.329861,0.052083,0.164931,35.890625,6.215278,0.032986,0.046875,576
1.0,371,0.102426,874.142857,0.145553,2.328841,2.514825,0.180593,0.021563,0.002695,0.830189,...,0.695418,1902.959569,0.172507,0.037736,0.221024,42.194070,7.229111,0.094340,0.010782,371
2.0,14,0.000000,790.571429,0.214286,2.285714,1.071429,0.142857,0.000000,0.000000,0.571429,...,0.500000,1268.071429,0.285714,0.000000,0.071429,63.500000,7.571429,0.000000,0.000000,14
3.0,138,0.094203,985.043478,0.065217,1.731884,2.188406,0.239130,0.021739,0.000000,0.724638,...,0.500000,1847.014493,0.217391,0.181159,0.217391,43.920290,8.181159,0.007246,0.007246,138
4.0,4,0.500000,639.750000,1.250000,4.250000,517.000000,1.500000,0.250000,0.500000,15.000000,...,4.750000,1078.000000,0.000000,0.000000,1.000000,22.500000,25.000000,0.000000,0.000000,4


In [24]:
ef = pd.read_csv("RNA_matrix_HealthyOnly.csv")
ef.drop(["file_id", "label"], "columns", inplace=True)
k_means = KMeans(n_clusters=5)
k_means.fit(ef) 

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [25]:
resultsHealth = k_means.labels_
print(np.unique(resultsHealth))
ef["clusterGroup"] = np.nan
dataLength = len(ef)
for i in range(dataLength):
    ef.iloc[i, ef.columns.get_loc("clusterGroup")] = resultsHealth[i]

[0 1 2 3 4]


In [26]:
ef_clusterData = ef.groupby(["clusterGroup"]).mean()
ef_clusterData["Count"] = pd.Series(ef.groupby(["clusterGroup"]).size())
ef_clusterData.insert(0, 'Count_', ef_clusterData.Count)
ef_clusterData

,Count_,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,ENSG00000265820.1,...,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1,Count
clusterGroup,,,,,,,,,,,,,,,,,,,,,
0.0,452,0.442478,11.298673,0.121681,0.044248,0.212389,0.011062,0.046460,0.000000,0.022124,...,0.466814,1214.238938,0.168142,0.004425,0.139381,26.163717,4.665929,0.004425,0.037611,452
1.0,73,0.342466,79.013699,0.068493,0.054795,0.547945,0.000000,0.027397,0.000000,0.041096,...,1.123288,2526.164384,0.027397,0.000000,0.082192,152.438356,8.931507,0.000000,0.041096,73
2.0,174,0.270115,36.385057,0.097701,0.040230,0.344828,0.000000,0.057471,0.000000,0.034483,...,0.844828,1738.839080,0.091954,0.005747,0.097701,70.574713,5.747126,0.000000,0.022989,174
3.0,30,0.300000,102.800000,0.033333,0.033333,0.533333,0.000000,0.000000,0.000000,0.000000,...,0.966667,1941.500000,0.033333,0.000000,0.033333,93.333333,8.866667,0.000000,0.000000,30
4.0,470,0.304255,7.597872,0.053191,0.042553,0.227660,0.004255,0.031915,0.002128,0.029787,...,0.608511,890.659574,0.174468,0.012766,0.168085,12.985106,3.255319,0.002128,0.034043,470


In [27]:
df_clusterData

,Count_,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,ENSG00000265820.1,...,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1,Count
clusterGroup,,,,,,,,,,,,,,,,,,,,,
0.0,576,0.145833,783.821181,0.097222,1.460069,2.406250,0.159722,0.039931,0.008681,0.666667,...,0.649306,1802.510417,0.329861,0.052083,0.164931,35.890625,6.215278,0.032986,0.046875,576
1.0,371,0.102426,874.142857,0.145553,2.328841,2.514825,0.180593,0.021563,0.002695,0.830189,...,0.695418,1902.959569,0.172507,0.037736,0.221024,42.194070,7.229111,0.094340,0.010782,371
2.0,14,0.000000,790.571429,0.214286,2.285714,1.071429,0.142857,0.000000,0.000000,0.571429,...,0.500000,1268.071429,0.285714,0.000000,0.071429,63.500000,7.571429,0.000000,0.000000,14
3.0,138,0.094203,985.043478,0.065217,1.731884,2.188406,0.239130,0.021739,0.000000,0.724638,...,0.500000,1847.014493,0.217391,0.181159,0.217391,43.920290,8.181159,0.007246,0.007246,138
4.0,4,0.500000,639.750000,1.250000,4.250000,517.000000,1.500000,0.250000,0.500000,15.000000,...,4.750000,1078.000000,0.000000,0.000000,1.000000,22.500000,25.000000,0.000000,0.000000,4


In [28]:
df.describe()

,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,ENSG00000265820.1,ENSG00000108947.4,...,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1,clusterGroup
count,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,...,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000,1103.000000
mean,0.124207,838.940163,0.115141,1.806890,4.264733,0.181324,0.031732,0.007253,0.779692,390.675431,...,0.659112,1832.454216,0.261106,0.062557,0.192203,39.317316,6.887579,0.049864,0.029012,0.751587
std,0.791277,742.571856,0.474750,3.586356,59.678880,0.769033,0.329837,0.120276,1.742358,508.336090,...,1.814397,748.780246,3.141953,0.751697,0.814597,45.114725,7.543826,0.794866,0.529589,1.003594
min,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,...,0.000000,263.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,347.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,105.000000,...,0.000000,1323.000000,0.000000,0.000000,0.000000,14.000000,2.000000,0.000000,0.000000,0.000000
50%,0.000000,648.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,228.000000,...,0.000000,1727.000000,0.000000,0.000000,0.000000,26.000000,5.000000,0.000000,0.000000,0.000000
75%,0.000000,1098.500000,0.000000,2.000000,3.000000,0.000000,0.000000,0.000000,1.000000,464.500000,...,1.000000,2245.500000,0.000000,0.000000,0.000000,47.000000,9.000000,0.000000,0.000000,1.000000
max,22.000000,11243.000000,8.000000,49.000000,1970.000000,12.000000,7.000000,3.000000,27.000000,6600.000000,...,37.000000,8005.000000,102.000000,19.000000,18.000000,389.000000,83.000000,24.000000,16.000000,4.000000


In [29]:
ef.describe()

,ENSG00000183169.6,ENSG00000100290.2,ENSG00000226765.1,ENSG00000241204.1,ENSG00000212135.1,ENSG00000254800.2,ENSG00000258516.1,ENSG00000206614.1,ENSG00000265820.1,ENSG00000108947.4,...,ENSG00000253842.1,ENSG00000125741.4,ENSG00000251610.1,ENSG00000253413.1,ENSG00000205333.5,ENSG00000259985.1,ENSG00000272735.1,ENSG00000238912.1,ENSG00000257787.1,clusterGroup
count,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,...,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000,1199.000000
mean,0.353628,19.900751,0.085905,0.043369,0.266055,0.005838,0.040033,0.000834,0.027523,214.728941,...,0.629691,1261.600500,0.147623,0.007506,0.138449,36.811510,4.634696,0.002502,0.033361,1.994162
std,0.997519,38.015518,0.432556,0.248106,0.593600,0.095644,0.277237,0.028880,0.178315,369.571912,...,1.543899,644.044563,0.712909,0.132185,0.512789,50.858797,4.790955,0.049979,0.201550,1.778878
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,249.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,28.000000,...,0.000000,797.500000,0.000000,0.000000,0.000000,8.500000,2.000000,0.000000,0.000000,0.000000
50%,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80.000000,...,0.000000,1091.000000,0.000000,0.000000,0.000000,18.000000,4.000000,0.000000,0.000000,2.000000
75%,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,220.000000,...,0.500000,1568.500000,0.000000,0.000000,0.000000,37.000000,6.000000,0.000000,0.000000,4.000000
max,10.000000,491.000000,5.000000,2.000000,4.000000,2.000000,4.000000,1.000000,2.000000,3649.000000,...,19.000000,5278.000000,10.000000,4.000000,4.000000,391.000000,39.000000,1.000000,2.000000,4.000000


In [30]:
GDCmean = df.mean()

In [31]:
GDCmean

ENSG00000183169.6        0.124207
ENSG00000100290.2      838.940163
ENSG00000226765.1        0.115141
ENSG00000241204.1        1.806890
ENSG00000212135.1        4.264733
ENSG00000254800.2        0.181324
ENSG00000258516.1        0.031732
ENSG00000206614.1        0.007253
ENSG00000265820.1        0.779692
ENSG00000108947.4      390.675431
ENSG00000255226.1        0.309157
ENSG00000230764.1        0.047144
ENSG00000251713.1        0.992747
ENSG00000220924.4        0.747960
ENSG00000231422.2        0.830462
ENSG00000263630.1        0.007253
ENSG00000260863.1        0.116954
ENSG00000264070.1       34.739801
ENSG00000229852.2      149.085222
ENSG00000233222.2        0.425204
ENSG00000264831.1        0.113327
ENSG00000249581.2        0.202176
ENSG00000253099.1        0.078876
ENSG00000271559.1        0.010879
ENSG00000202431.1        1.230281
ENSG00000201524.1        0.549411
ENSG00000227076.1        9.240254
ENSG00000228703.1        0.000000
ENSG00000205790.1       21.087942
ENSG0000022820

In [32]:
GTexmean = ef.mean()

In [33]:
GTexmean

ENSG00000183169.6        0.353628
ENSG00000100290.2       19.900751
ENSG00000226765.1        0.085905
ENSG00000241204.1        0.043369
ENSG00000212135.1        0.266055
ENSG00000254800.2        0.005838
ENSG00000258516.1        0.040033
ENSG00000206614.1        0.000834
ENSG00000265820.1        0.027523
ENSG00000108947.4      214.728941
ENSG00000255226.1        1.221018
ENSG00000230764.1        0.000000
ENSG00000251713.1        0.046706
ENSG00000220924.4        0.065054
ENSG00000231422.2        0.874062
ENSG00000263630.1        0.005004
ENSG00000260863.1        0.130942
ENSG00000264070.1       38.675563
ENSG00000229852.2       91.982485
ENSG00000233222.2        1.433695
ENSG00000264831.1        0.001668
ENSG00000249581.2        0.160133
ENSG00000253099.1        0.005004
ENSG00000271559.1        0.007506
ENSG00000202431.1        0.000834
ENSG00000201524.1        0.030859
ENSG00000227076.1        4.590492
ENSG00000228703.1       15.069224
ENSG00000205790.1      105.152627
ENSG0000022820

In [34]:
type(GDCmean)

pandas.core.series.Series

In [44]:
GDCmeanList = GDCmean.tolist()
GTexmeanList = GTexmean.tolist()
cols = df.columns.tolist()

In [55]:
GTexmeanList

[0.353628023352794,
 19.90075062552127,
 0.08590492076730609,
 0.043369474562135114,
 0.26605504587155965,
 0.005838198498748957,
 0.040033361134278564,
 0.0008340283569641367,
 0.027522935779816515,
 214.72894078398664,
 1.2210175145954962,
 0.0,
 0.04670558798999166,
 0.06505421184320268,
 0.8740617180984154,
 0.0050041701417848205,
 0.13094245204336946,
 38.67556296914095,
 91.98248540450375,
 1.4336947456213511,
 0.0016680567139282735,
 0.16013344453711426,
 0.0050041701417848205,
 0.007506255212677231,
 0.0008340283569641367,
 0.030859049207673062,
 4.590492076730609,
 15.069224353628023,
 105.15262718932443,
 1.432026688907423,
 0.0008340283569641367,
 0.09674728940783987,
 6.635529608006673,
 0.058381984987489574,
 0.02585487906588824,
 0.7848206839032527,
 0.08757297748123437,
 0.2677231025854879,
 0.0884070058381985,
 127.41534612176814,
 11.08090075062552,
 4.703085904920767,
 0.8557130942452044,
 0.7447873227689742,
 3.999165971643036,
 3.2543786488740616,
 0.046705587989991

In [69]:
featureColumns = []
for i in range(len(GDCmeanList)):
    if(abs(GDCmeanList[i] - GTexmeanList[i]) > 10000):
        featureColumns.append(cols[i])

len(featureColumns)

76

In [59]:
featureColumns

['ENSG00000100290.2',
 'ENSG00000108947.4',
 'ENSG00000019995.6',
 'ENSG00000108239.8',
 'ENSG00000189114.6',
 'ENSG00000261468.1',
 'ENSG00000169018.5',
 'ENSG00000163064.6',
 'ENSG00000234840.1',
 'ENSG00000167395.10',
 'ENSG00000178502.5',
 'ENSG00000253755.1',
 'ENSG00000163866.8',
 'ENSG00000235649.2',
 'ENSG00000183722.7',
 'ENSG00000124098.9',
 'ENSG00000167771.5',
 'ENSG00000181634.7',
 'ENSG00000107341.4',
 'ENSG00000127554.12',
 'ENSG00000179941.6',
 'ENSG00000139329.4',
 'ENSG00000231768.1',
 'ENSG00000139193.3',
 'ENSG00000121621.6',
 'ENSG00000128268.11',
 'ENSG00000254973.1',
 'ENSG00000145779.7',
 'ENSG00000140379.7',
 'ENSG00000125848.9',
 'ENSG00000214975.4',
 'ENSG00000203635.2',
 'ENSG00000182916.7',
 'ENSG00000174567.7',
 'ENSG00000163220.10',
 'ENSG00000173281.4',
 'ENSG00000168795.4',
 'ENSG00000241294.1',
 'ENSG00000096401.7',
 'ENSG00000268686.1',
 'ENSG00000228253.1',
 'ENSG00000248275.1',
 'ENSG00000168303.6',
 'ENSG00000239306.4',
 'ENSG00000172215.5',
 'ENSG